最简化仿真代码（非训练）

In [ ]:
import sys
import os
import numpy as np
import torch as th
from math import *
# 获取project目录
def get_current_file_dir():
    # 判断是否在 Jupyter Notebook 环境
    try:
        shell = get_ipython().__class__.__name__  # ← 误报，不用管
        if shell == 'ZMQInteractiveShell':  # Jupyter Notebook 或 JupyterLab
            # 推荐用 os.getcwd()，指向启动 Jupyter 的目录
            return os.getcwd()
        else:  # 其他 shell
            return os.path.dirname(os.path.abspath(__file__))
    except NameError:
        # 普通 Python 脚本
        return os.path.dirname(os.path.abspath(__file__))

current_dir = get_current_file_dir()
sys.path.append(os.path.dirname(current_dir))
from Envs.UAVmodel6d import UAVModel
from Math_calculates.CartesianOnEarth import NUE2LLH, LLH2NUE

# class height_track

o00 = np.array([118, 30])  # 地理原点的经纬
DEFAULT_RED_BIRTH_STATE = {'position': np.array([-38000.0, 8000.0, 0.0]),
                               'psi': 0
                               }
dt_move=0.02
UAV = UAVModel(dt=dt_move)
UAV.ammo = 0
UAV.id = 1
UAV.red = True
UAV.blue = False
UAV.label = "red"
UAV.color = np.array([1, 0, 0])
# 红方出生点
UAV.pos_ = DEFAULT_RED_BIRTH_STATE['position']
UAV.speed = 300  # (UAV.speed_max - UAV.speed_min) / 2
speed = UAV.speed
UAV.psi = DEFAULT_RED_BIRTH_STATE['psi']
UAV.theta = 0 * pi / 180
UAV.gamma = 0 * pi / 180
UAV.vel_ = UAV.speed * np.array([cos(UAV.theta) * cos(UAV.psi),
                                    sin(UAV.theta),
                                    cos(UAV.theta) * sin(UAV.psi)])
lon_uav, lat_uav, h_uav = NUE2LLH(UAV.pos_[0], UAV.pos_[1], UAV.pos_[2], lon_o=o00[0], lat_o=o00[1], h_o=0)
UAV.reset(lon0=lon_uav, lat0=lat_uav, h0=h_uav, v0=UAV.speed, psi0=UAV.psi, phi0=UAV.gamma,
            theta0=UAV.theta, o00=o00)

action = [0,0,0]

target_height = 3000 + (action[0] + 1) / 2 * (10000 - 3000)  # 高度使用绝对数值
delta_heading = action[1]  # 相对方位(弧度)
target_speed = 170 + (action[2] + 1) / 2 * (544 - 170)  # 速度使用绝对数值
# print('target_height',target_height)
# for i in range(int(self.dt // dt_move)):
t_last = 60

tacview_show = 1

if tacview_show:
    from Visualize.tacview_visualize import *
    tacview = Tacview()

for i in range(int(t_last//dt_move)):
    current_t = i*dt_move
    UAV.move(target_height, delta_heading, target_speed)
    loc_r = [UAV.lon, UAV.lat, UAV.alt]
    if tacview_show:
        data_to_send = ''
        data_to_send += "#%.2f\n%s,T=%.6f|%.6f|%.6f|%.6f|%.6f|%.6f,Name=F16,Color=Red\n" % (
                float(current_t), UAV.id, loc_r[0], loc_r[1], loc_r[2], UAV.phi * 180 / pi, UAV.theta * 180 / pi,
                UAV.psi * 180 / pi)
    if tacview_show:
            tacview.send_data_to_client(data_to_send)

